<a href="https://colab.research.google.com/github/camlet0630/ESG_ranking_system/blob/main/nn_v0_pure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
import os
import gc
import random
from collections import Counter
from tqdm import tqdm
from IPython.display import display
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda', 0) if torch.cuda.is_available() else 'cpu'

In [ ]:
# 載入 embedding_tensor.pt & label_data.csv
embedding_tensor = torch.load("/content/embedding_tensor.pt")
print(embedding_tensor.shape)
label_data = pd.read_csv("/content/label_data.csv")

# config
config = {"seed": 168,
          "learning_rate": 3e-4,
          "batch_size": 64, #128 256訓練不了
          "epochs": 300}

def set_random_seed(seed, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)
set_random_seed(config["seed"])

torch.Size([1805, 768])


# Func


#### Evaluation Metrics

In [ ]:
# 函數: 計算 Accuracy
def ACCscore_m_label(y_true, pred):
    accuracy_l = [ans.all() for ans in (pred == y_true)]
    accuracy = np.array(accuracy_l).mean()
    return accuracy

# 函數: 計算 Precision, Recall
def PRscore_m_label(y_true, pred):
    hit_matrix = np.zeros_like(pred)
    hit_matrix[np.where((pred == y_true) & (y_true > 0))] = 1
    tp = hit_matrix.sum(axis=1)
    pred_sum = pred.sum(axis=1)
    true_sum = y_true.sum(axis=1)
    precision_l = []
    recall_l = []
    for ix in range(tp.shape[0]):
        precision_score = (1.0 if true_sum[ix] == 0 else 0.0) if pred_sum[ix] == 0 else tp[ix]/pred_sum[ix]
        recall_score = (1.0 if pred_sum[ix] == 0 else 0.0) if true_sum[ix] == 0 else tp[ix]/true_sum[ix]
        precision_l.append(precision_score)
        recall_l.append(recall_score)
    precision = np.array(precision_l).mean()
    recall = np.array(recall_l).mean()
    return precision, recall

# 函數: 計算 F1-score
def f1(precision, recall):
    f1_score = 0
    if (precision + recall) !=0:
        f1_score = (2 * precision * recall) / (precision + recall)
    return f1_score

# 函數: 一次同時計算所有評估指標
def score(y_true, pred):
    accuracy = ACCscore_m_label(y_true, pred)
    precision, recall = PRscore_m_label(y_true, pred)
    f1_score = f1(precision, recall)
    return accuracy, precision, recall, f1_score

#### Calculate Class Weight

In [ ]:
# Function: Class weight calculation
def get_class_weight(df, labels):
    # 計算每個類別的「正樣本數量」，儲存成 class_pos_count 列表
    class_pos_count = np.zeros(len(labels))
    # 計算每個類別的「負樣本數量」，儲存成 class_neg_count 列表
    class_neg_count = np.zeros(len(labels))
    # 遞迴
    for i in range(len(labels)):
        label = labels[i]
        positive, negative = df[label].value_counts()[1.0], df[label].value_counts()[0.0]
        class_pos_count[i], class_neg_count[i] = positive, negative
    # 計算「正樣本權重」
    class_pos_weight = np.ones_like(class_pos_count)
    for cdx, (pos_count, neg_count) in enumerate(zip(class_pos_count, class_neg_count)):
        total_count = pos_count + neg_count
        postive_weights = total_count/(2*pos_count)
        negtive_weights = total_count/(2*neg_count)
        class_pos_weight[cdx] = (postive_weights/negtive_weights)
    return torch.as_tensor(class_pos_weight, dtype=torch.float)

#### Print out Class Weight

In [ ]:
def print_class_weight(task_label):
    print("<<各類別資料數>>")
    print(task_label.sum())
    label_list = task_label.columns
    class_weight = get_class_weight(task_label, label_list)
    print()
    print("<<目標變數與權重對應>>")
    print(f"目標變數共有 {len(label_list)} 類")
    for c, w in zip(label_list, class_weight):
        print(f"{c}: {round(float(w), 2)}")

#### Build Class: Dataset

In [ ]:
class Dataset(Dataset):
    def __init__(self, embeddings, label_list):
        self.embeddings = embeddings
        self.labels = label_list.to_numpy()

    def __getLabels__(self):
        return (self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Modeling

#### Model Architecture
- 實現一個神經網絡模型。該模型包含了兩層隱藏層架構並使用 dropout 機制

In [ ]:
print(nn.Module)

<class 'torch.nn.modules.module.Module'>


In [ ]:
class Classifier(nn.Module): # multitask + teacherForcing() + posiNega
    def __init__(self, output_size_1=3, output_size_2=22, output_size_3=3, dropout_rate=0.5):
        super(Classifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(768, 256) # Stage_0
        self.linear_2 = nn.Linear(256, 64) # Stage_0
        self.linear_3 = nn.Linear(64, output_size_1) # Stage_1_rough_NN (得到 rough_output)
        self.linear_4 = nn.Linear(64, 32)  # Stage_1_noCat_NN（得到 noCat_output）
        self.linear_5 = nn.Linear(32+output_size_1, output_size_2) # Stage_2_NN（得到 precise_output）
        self.output_layers = nn.ModuleList([nn.Sequential(nn.Linear(32+output_size_1, output_size_3), nn.Softmax) for _ in range(output_size_2)]) # 22 個 Stage_2.0_NN

    def forward(self, embeddings, y_rough_true, alpha):
        shared_output = self.linear_1(embeddings)
        shared_output = self.relu(shared_output)
        shared_output = self.linear_2(shared_output)
        shared_output = self.relu(shared_output)
        shared_output = self.dropout(shared_output)
        # 已經走完 Stage_0 得到 shared_output(64)
        rough_output = self.linear_3(shared_output)
        noCat_output = self.linear_4(shared_output)
        rept_rough_output = alpha * y_rough_true + (1-alpha) * rough_output # 當 epoch > 10 開始 alpha 1->0
        withCat_output = torch.cat((rept_rough_output, noCat_output), axis=1)
        withCat_output = self.relu(withCat_output)
        withCat_output = self.dropout(withCat_output)
        # 已經 concat 完得到 withCat_output
        # precise_output = self.linear_5(withCat_output)
        precise_output = [output_layer(withCat_output) for output_layer in self.output_layers] # 22x3
        return rough_output, precise_output

#### Function for model training

In [ ]:
def train(model, class_weight, train_set, val_set, patience=100):
    train_loader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=config["batch_size"], shuffle=False)
    # Loss Function
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weight)

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])

    # Putting the model on the GPU to run
    model = model.to(device)
    criterion = criterion.to(device)

    val_best_f1 = float(-np.inf)
    iteration_num = 0
    epoch_num = 0

    epochs = config["epochs"]

    while True:
        if iteration_num >= patience:
            break

        # Train and save training results
        loss_list = []
        accuracy_list = []
        precision_list = []
        recall_list = []
        f1_list = []

        # Set the model to the "training" state (Layers such as Dropout will be triggered only)
        model.train()

        for train_embeddings, train_labels in tqdm(train_loader):

            optimizer.zero_grad()

            train_embeddings = train_embeddings.to(device)
            train_labels = train_labels.to(device)

            output = model(train_embeddings)

            batch_loss = criterion(output, train_labels.float())

            # Back propagation
            batch_loss.backward()
            optimizer.step()

            output = output.cpu().detach()
            pred = (output > 0).cpu()
            y_true = (train_labels > 0).cpu()

            # Calculate the performance of each batch and save it
            loss = batch_loss.item()
            accuracy, precision, recall, f1_score= score(y_true, pred)

            loss_list.append(loss)
            accuracy_list.append(accuracy)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1_score)

        # Calculate the performance of each epoch and save it
        train_loss = np.array(loss_list).mean()
        train_accuracy = np.array(accuracy_list).mean()
        train_precision = np.array(precision_list).mean()
        train_recall = np.array(recall_list).mean()
        train_f1 = np.array(f1_list).mean()


        # Evaluate the current model performance using validate dataset
        loss_list = []
        accuracy_list = []
        precision_list = []
        recall_list = []
        pred_list = []
        f1_list = []

        with torch.no_grad():
            model.eval()
            for val_embeddings, val_labels in tqdm(val_loader):

                val_embeddings = val_embeddings.to(device)
                val_labels = val_labels.to(device)

                output = model(val_embeddings)

                batch_loss = criterion(output, val_labels.float())

                output = output.cpu()
                pred = (output > 0).cpu()
                sum_pred = torch.sum(pred, dim=1)
                non_label_case_idx = (sum_pred < 1).nonzero()

                if non_label_case_idx.shape[0] != 0:
                    non_label_case = output[non_label_case_idx]
                    max_col = torch.argmax(non_label_case, dim=-1)
                    raw = non_label_case_idx.reshape(-1,1)
                    pred_np = pred.numpy()
                    for idx in range(raw.shape[0]):
                        pred_np[raw[idx]][max_col[idx]] = True
                    pred = torch.from_numpy(pred_np)
                pred_list.extend(pred.numpy().astype('float32'))
                y_true = (val_labels > 0).cpu()

                # Calculate the performance of each batch and save it
                loss = batch_loss.item()
                accuracy, precision, recall, f1_score = score(y_true, pred)

                loss_list.append(loss)
                accuracy_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                f1_list.append(f1_score)

            # Calculate the performance of each epoch and save it
            val_loss = np.array(loss_list).mean()
            val_accuracy = np.array(accuracy_list).mean()
            val_precision = np.array(precision_list).mean()
            val_recall = np.array(recall_list).mean()
            val_f1 = np.array(f1_list).mean()


        if val_f1 > val_best_f1:
            ## val result
            val_best_f1 = val_f1
            val_best_loss = val_loss
            val_best_accuracy = val_accuracy
            val_best_precision = val_precision
            val_best_recall = val_recall
            ## train result
            train_best_f1 = train_f1
            train_best_loss = train_loss
            train_best_accuracy = train_accuracy
            train_best_precision = train_precision
            train_best_recall = train_recall
            ## other result
            best_model = model
            iteration_num = 0
            best_pred_list = pred_list
        else:
            iteration_num += 1

        epoch_num += 1
        # Print out the results of each Epoch
        print(f'Epochs: {epoch_num + 1} || Train Loss: {train_loss: .3f}, Accuracy: {train_accuracy: .3f}, Precision: {train_precision: .3f}, Recall:{train_recall: .3f}, F1: {train_f1: .3f} || Val Loss: {val_loss: .3f}, Accuracy: {val_accuracy: .3f}, Precision: {val_precision: .3f}, Recall:{val_recall: .3f}, Val F1: {val_f1: .3f} || Best F1: {val_best_f1: .3f}, Iter: {iteration_num: .0f}')

    train_result = [train_best_loss, train_best_accuracy, train_best_precision,
                    train_best_recall, train_best_f1]
    val_result = [val_best_loss, val_best_accuracy, val_best_precision,
                  val_best_recall, val_best_f1]

    return model, train_result, val_result, best_pred_list

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### 10-Fold Cross-Validation

In [ ]:
def kfold_train_m_label(embedding_tensor, task_label, output_size, nfold=10):

    # 10 fold
    skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=config["seed"])
    kfold_result = pd.DataFrame(columns=['Fold', 'train_loss', 'train_accuracy', 'train_precision',
                                    'train_recall', 'train_f1', 'val_loss', 'val_accuracy',
                                    'val_precision', 'val_recall', 'val_f1'])

    counts_true = pd.DataFrame(columns = ["num_label"])
    counts_pred = pd.DataFrame(columns = ["num_label"])

    pred_col_name = ["idx"]
    for o_idx in range(output_size):
        pred_col_name.append(f"{o_idx}")
    pred_result = pd.DataFrame(columns=pred_col_name)

    for fold_i, (train_fold, test_fold) in enumerate(skf.split(embedding_tensor, task_label.to_numpy().argmax(1))):

        print(f"Fold: {fold_i} | Train shape: {len(train_fold)} | Val shape: {len(test_fold)}")

        ## Preparation of training and validation datasets
        embedding_train, embedding_val = embedding_tensor[train_fold], embedding_tensor[test_fold]
        label_train, label_val = task_label.iloc[train_fold,:], task_label.iloc[test_fold,:]

        class_weight = print_class_weight(label_train)

        ## Actual number of labels
        actual_label_count = pd.DataFrame(sorted(Counter(label_val.sum(axis=1)).items()),
                                        columns = ["num_label", f"true_{fold_i}"])
        counts_true = pd.merge(counts_true, actual_label_count, how="outer", on = "num_label")

        ## Convert to Dataset data type
        train_set = Dataset(embedding_train, label_train)
        val_set = Dataset(embedding_val, label_val)

        print(f"Train 各類別資料量:", np.sum(train_set.__getLabels__(), axis=0))
        print(f"Val 各類別資料量:", np.sum(val_set.__getLabels__(), axis=0))
        model = Classifier(output_size=output_size)

        ## Model Training
        model, train_result, val_result, pred_list = train(model, class_weight, train_set, val_set)

        ## Record model evaluation results
        kfold_result = kfold_result.append([{"Fold": fold_i,
                                        "train_loss":train_result[0],
                                        "train_accuracy":train_result[1],
                                        "train_precision":train_result[2],
                                        "train_recall":train_result[3],
                                        "train_f1":train_result[4],
                                        "val_loss": val_result[0],
                                        "val_accuracy": val_result[1],
                                        "val_precision": val_result[2],
                                        "val_recall": val_result[3],
                                        "val_f1": val_result[4]}])


        ## Prediction results
        pred_list = np.array(pred_list)

        new_record = {}
        new_record['idx'] = test_fold

        for o_idx in range(1, len(pred_col_name)):
            new_record[pred_col_name[o_idx]] = pred_list[:,o_idx-1]

        pred_result = pred_result.append(pd.DataFrame(new_record), ignore_index=True)
        print("-"*100)

        ## Predicted number of labels
        pred_label_count = pd.DataFrame(sorted(Counter(pred_list.sum(axis=1)).items()),
                          columns=["num_label", f"pred_{fold_i}"])
        counts_pred = pd.merge(counts_pred, pred_label_count, how="outer", on = "num_label")


    return kfold_result, pred_result, counts_true, counts_pred

# Mainfunction

In [ ]:
kfold_result_DL, pred_result_DL, counts_true_DL, counts_pred_DL = kfold_train_m_label(embedding_tensor,
                                                                                      label_data,
                                                                                      label_data.shape[1])

# Evaluation

In [ ]:
pred_result_DL = pred_result_DL.sort_values("idx")
pred_result_DL.reset_index(inplace=True, drop=True)
display(kfold_result_DL)
display(kfold_result_DL.mean()[2:])

NameError: ignored

# Evaluation Metrics_ new ver.
Multilabel Classification 是要對每一個 class 算分數


可參考: https://towardsdatascience.com/evaluating-multi-label-classifiers-a31be83da6ea

In [ ]:
# 函數: 計算 Accuracy
def ACCscore_m_label(y_true, pred):
    accuracy_l = (pred == y_true)
    accuracy = np.array(accuracy_l).mean()
    return accuracy

# 函數: 計算 Precision, Recall
def PRscore_m_label(y_true, pred):
    hit_matrix = np.zeros_like(pred)
    hit_matrix[np.where((pred == y_true) & (y_true > 0))] = 1
    tp = hit_matrix.sum(axis=1)
    pred_sum = pred.sum(axis=1)
    true_sum = y_true.sum(axis=1)
    precision_l = []
    recall_l = []
    for ix in range(tp.shape[0]):
        precision_score = (1.0 if true_sum[ix] == 0 else 0.0) if pred_sum[ix] == 0 else tp[ix]/pred_sum[ix]
        recall_score = (1.0 if pred_sum[ix] == 0 else 0.0) if true_sum[ix] == 0 else tp[ix]/true_sum[ix]
        precision_l.append(precision_score)
        recall_l.append(recall_score)
    precision = np.array(precision_l).mean()
    recall = np.array(recall_l).mean()
    return precision, recall

# 函數: 計算 F1-score
def f1(precision, recall):
    f1_score = 0
    if (precision + recall) !=0:
        f1_score = (2 * precision * recall) / (precision + recall)
    return f1_score

# 函數: 一次同時計算所有評估指標
def score(y_true, pred):
    accuracy = ACCscore_m_label(y_true, pred)
    precision, recall = PRscore_m_label(y_true, pred)
    f1_score = f1(precision, recall)
    return accuracy, precision, recall, f1_score

# Function for model training

In [ ]:
def train(model, class_weight, train_set, val_set, patience=100):
    train_loader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=config["batch_size"], shuffle=False)
    # Loss Function
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weight)

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])

    # Putting the model on the GPU to run
    model = model.to(device)
    criterion = criterion.to(device)

    val_best_f1 = float(-np.inf)
    iteration_num = 0
    epoch_num = 0

    epochs = config["epochs"]

    while True:
        if iteration_num >= patience:
            break

        # Train and save training results
        loss_list = []
        accuracy_list = []
        precision_list = []
        recall_list = []
        f1_list = []

        # Set the model to the "training" state (Layers such as Dropout will be triggered only)
        model.train()

        for train_embeddings, train_labels in tqdm(train_loader):

            optimizer.zero_grad()

            train_embeddings = train_embeddings.to(device)
            train_labels = train_labels.to(device)

            output = model(train_embeddings)

            batch_loss = criterion(output, train_labels.float())

            # Back propagation
            batch_loss.backward()
            optimizer.step()

            output = output.cpu().detach()
            pred = (output > 0).cpu()
            y_true = (train_labels > 0).cpu()

            # Calculate the performance of each batch and save it
            loss = batch_loss.item()
            accuracy, precision, recall, f1_score= score(y_true, pred)

            loss_list.append(loss)
            accuracy_list.append(accuracy)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1_score)

        # Calculate the performance of each epoch and save it
        train_loss = np.array(loss_list).mean()
        train_accuracy = np.array(accuracy_list).mean()
        train_precision = np.array(precision_list).mean()
        train_recall = np.array(recall_list).mean()
        train_f1 = np.array(f1_list).mean()


        # Evaluate the current model performance using validate dataset
        loss_list = []
        accuracy_list = []
        precision_list = []
        recall_list = []
        pred_list = []
        f1_list = []

        with torch.no_grad():
            model.eval()
            for val_embeddings, val_labels in tqdm(val_loader):

                val_embeddings = val_embeddings.to(device)
                val_labels = val_labels.to(device)

                output = model(val_embeddings)

                batch_loss = criterion(output, val_labels.float())

                output = output.cpu()
                pred = (output > 0).cpu()
                sum_pred = torch.sum(pred, dim=1)
                non_label_case_idx = (sum_pred < 1).nonzero()

                if non_label_case_idx.shape[0] != 0:
                    non_label_case = output[non_label_case_idx]
                    max_col = torch.argmax(non_label_case, dim=-1)
                    raw = non_label_case_idx.reshape(-1,1)
                    pred_np = pred.numpy()
                    for idx in range(raw.shape[0]):
                        pred_np[raw[idx]][max_col[idx]] = True
                    pred = torch.from_numpy(pred_np)
                pred_list.extend(pred.numpy().astype('float32'))
                y_true = (val_labels > 0).cpu()

                # Calculate the performance of each batch and save it
                loss = batch_loss.item()
                accuracy, precision, recall, f1_score = score(y_true, pred)

                loss_list.append(loss)
                accuracy_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                f1_list.append(f1_score)

            # Calculate the performance of each epoch and save it
            val_loss = np.array(loss_list).mean()
            val_accuracy = np.array(accuracy_list).mean()
            val_precision = np.array(precision_list).mean()
            val_recall = np.array(recall_list).mean()
            val_f1 = np.array(f1_list).mean()


        if val_f1 > val_best_f1:
            ## val result
            val_best_f1 = val_f1
            val_best_loss = val_loss
            val_best_accuracy = val_accuracy
            val_best_precision = val_precision
            val_best_recall = val_recall
            ## train result
            train_best_f1 = train_f1
            train_best_loss = train_loss
            train_best_accuracy = train_accuracy
            train_best_precision = train_precision
            train_best_recall = train_recall
            ## other result
            best_model = model
            iteration_num = 0
            best_pred_list = pred_list
        else:
            iteration_num += 1

        epoch_num += 1
        # Print out the results of each Epoch
        print(f'Epochs: {epoch_num + 1} || Train Loss: {train_loss: .3f}, Accuracy: {train_accuracy: .3f}, Precision: {train_precision: .3f}, Recall:{train_recall: .3f}, F1: {train_f1: .3f} || Val Loss: {val_loss: .3f}, Accuracy: {val_accuracy: .3f}, Precision: {val_precision: .3f}, Recall:{val_recall: .3f}, Val F1: {val_f1: .3f} || Best F1: {val_best_f1: .3f}, Iter: {iteration_num: .0f}')

    train_result = [train_best_loss, train_best_accuracy, train_best_precision,
                    train_best_recall, train_best_f1]
    val_result = [val_best_loss, val_best_accuracy, val_best_precision,
                  val_best_recall, val_best_f1]

    return model, train_result, val_result, best_pred_list

In [ ]:
def train(model, class_weight, train_set, val_set, patience=100):
    train_loader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=config["batch_size"], shuffle=False)
    # Loss Function
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weight)

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])

    # Putting the model on the GPU to run
    model = model.to(device)
    criterion = criterion.to(device)

    val_best_f1 = float(-np.inf)
    iteration_num = 0
    epoch_num = 0

    epochs = config["epochs"]

    while True:
        if iteration_num >= patience:
            break

        # Train and save training results
        loss_list_E = []
        accuracy_list_E = []
        precision_list_E = []
        recall_list_E = []
        f1_list_E = []

        loss_list_S = []
        accuracy_list_S = []
        precision_list_S = []
        recall_list_S = []
        f1_list_S = []

        loss_list_G = []
        accuracy_list_G = []
        precision_list_G = []
        recall_list_G = []
        f1_list_G = []

        # Set the model to the "training" state (Layers such as Dropout will be triggered only)
        model.train()

        for train_embeddings, train_labels in tqdm(train_loader):

            optimizer.zero_grad()

            train_embeddings = train_embeddings.to(device)
            train_labels = train_labels.to(device)

            output = model(train_embeddings)

            batch_loss = criterion(output, train_labels.float())

            # Back propagation
            batch_loss.backward()
            optimizer.step()

            output = output.cpu().detach()
            pred = (output > 0).cpu()
            y_true = (train_labels > 0).cpu()

            # Calculate the performance of each batch and save it
            loss = batch_loss.item()

            # E
            accuracy, precision, recall, f1_score= score(y_true[0], pred[0])
            loss_list_E.append(loss)
            accuracy_list_E.append(accuracy)
            precision_list_E.append(precision)
            recall_list_E.append(recall)
            f1_list_E.append(f1_score)

            # S
            accuracy, precision, recall, f1_score= score(y_true[1], pred[1])
            loss_list_S.append(loss)
            accuracy_list_S.append(accuracy)
            precision_list_S.append(precision)
            recall_list_S.append(recall)
            f1_list_S.append(f1_score)

            # G
            accuracy, precision, recall, f1_score= score(y_true[2], pred[2])
            loss_list_G.append(loss)
            accuracy_list_G.append(accuracy)
            precision_list_G.append(precision)
            recall_list_G.append(recall)
            f1_list_G.append(f1_score)




        # Calculate the performance of each epoch and save it
        train_loss_E = np.array(loss_list_E).mean()
        train_accuracy_E = np.array(accuracy_list_E).mean()
        train_precision_E = np.array(precision_list_E).mean()
        train_recall_E = np.array(recall_list_E).mean()
        train_f1_E = np.array(f1_list_E).mean()

        # S
        train_loss_S = np.array(loss_list_S).mean()
        train_accuracy_S = np.array(accuracy_list_S).mean()
        train_precision_S = np.array(precision_list_S).mean()
        train_recall_S = np.array(recall_list_S).mean()
        train_f1_S = np.array(f1_list_S).mean()

        # G
        train_loss_G = np.array(loss_list_G).mean()
        train_accuracy_G = np.array(accuracy_list_G).mean()
        train_precision_G = np.array(precision_list_G).mean()
        train_recall_G = np.array(recall_list_G).mean()
        train_f1_G = np.array(f1_list_G).mean()



        # Evaluate the current model performance using validate dataset
        loss_list_E = []
        accuracy_list_E = []
        precision_list_E = []
        recall_list_E = []
        f1_list_E = []

        loss_list_S = []
        accuracy_list_S = []
        precision_list_S = []
        recall_list_S = []
        f1_list_S = []

        loss_list_G = []
        accuracy_list_G = []
        precision_list_G = []
        recall_list_G = []
        f1_list_G = []

        with torch.no_grad():
            model.eval()
            for val_embeddings, val_labels in tqdm(val_loader):

                val_embeddings = val_embeddings.to(device)
                val_labels = val_labels.to(device)

                output = model(val_embeddings)

                batch_loss = criterion(output, val_labels.float())

                output = output.cpu()
                pred = (output > 0).cpu()
                sum_pred = torch.sum(pred, dim=1)
                non_label_case_idx = (sum_pred < 1).nonzero()

                if non_label_case_idx.shape[0] != 0:
                    non_label_case = output[non_label_case_idx]
                    max_col = torch.argmax(non_label_case, dim=-1)
                    raw = non_label_case_idx.reshape(-1,1)
                    pred_np = pred.numpy()
                    for idx in range(raw.shape[0]):
                        pred_np[raw[idx]][max_col[idx]] = True
                    pred = torch.from_numpy(pred_np)
                pred_list.extend(pred.numpy().astype('float32'))
                y_true = (val_labels > 0).cpu()

                # Calculate the performance of each batch and save it
                loss = batch_loss.item()

                # E
                accuracy, precision, recall, f1_score= score(y_true[0], pred[0])
                loss_list_E.append(loss)
                accuracy_list_E.append(accuracy)
                precision_list_E.append(precision)
                recall_list_E.append(recall)
                f1_list_E.append(f1_score)

                # S
                accuracy, precision, recall, f1_score= score(y_true[1], pred[1])
                loss_list_S.append(loss)
                accuracy_list_S.append(accuracy)
                precision_list_S.append(precision)
                recall_list_S.append(recall)
                f1_list_S.append(f1_score)

                # G
                accuracy, precision, recall, f1_score= score(y_true[2], pred[2])
                loss_list_G.append(loss)
                accuracy_list_G.append(accuracy)
                precision_list_G.append(precision)
                recall_list_G.append(recall)
                f1_list_G.append(f1_score)

            # Calculate the performance of each epoch and save it
            val_loss_E = np.array(loss_list_E).mean()
            val_accuracy_E = np.array(accuracy_list_E).mean()
            val_precision_E = np.array(precision_list_E).mean()
            val_recall_E = np.array(recall_list_E).mean()
            val_f1_E = np.array(f1_list_E).mean()

            # S
            val_loss_S = np.array(loss_list_S).mean()
            val_accuracy_S = np.array(accuracy_list_S).mean()
            valn_precision_S = np.array(precision_list_S).mean()
            valn_recall_S = np.array(recall_list_S).mean()
            val_f1_S = np.array(f1_list_S).mean()

            # G
            val_loss_G = np.array(loss_list_G).mean()
            val_accuracy_G = np.array(accuracy_list_G).mean()
            val_precision_G = np.array(precision_list_G).mean()
            val_recall_G = np.array(recall_list_G).mean()
            val_f1_G = np.array(f1_list_G).mean()


        if val_f1_E + val_f1_S + val_f1_G > val_best_f1_E + val_best_f1_S + val_best_f1_G:
            ## val result
            val_best_f1_E = val_f1_E
            val_best_loss_E = val_loss_E
            val_best_accuracy_E = val_accuracy_E
            val_best_precision_E = val_precision_E
            val_best_recall_E = val_recall_E
            val_best_f1_S = val_f1_S
            val_best_loss_S = val_loss_S
            val_best_accuracy_S = val_accuracy_S
            val_best_precision_S = val_precision_S
            val_best_recall_S = val_recall_S
            val_best_f1_G = val_f1_G
            val_best_loss_G = val_loss_G
            val_best_accuracy_G = val_accuracy_G
            val_best_precision_G = val_precision_G
            val_best_recall_G = val_recall_G
            ## train result
            train_best_f1_E = train_f1_E
            train_best_loss_E = train_loss_E
            train_best_accuracy_E = train_accuracy_E
            train_best_precision_E = train_precision_E
            train_best_recall_E = train_recall_E
            train_best_f1_S = train_f1_S
            train_best_loss_S = train_loss_S
            train_best_accuracy_S = train_accuracy_S
            train_best_precision_S = train_precision_S
            train_best_recall_S = train_recall_S
            train_best_f1_G = train_f1_G
            train_best_loss_G = train_loss_G
            train_best_accuracy_G = train_accuracy_G
            train_best_precision_G = train_precision_G
            train_best_recall_G = train_recall_G
            ## other result
            best_model = model
            iteration_num = 0
            best_pred_list = pred_list
        else:
            iteration_num += 1

        epoch_num += 1
        # Print out the results of each Epoch
        # print?
        print(f'Epochs: {epoch_num + 1} || E Train Loss: {train_loss_E: .3f}, Accuracy: {train_accuracy_E: .3f}, Precision: {train_precision_E: .3f}, Recall:{train_recall_E: .3f}, F1: {train_f1_E: .3f} || Val Loss: {val_loss_E: .3f}, Accuracy: {val_accuracy_E: .3f}, Precision: {val_precision_E: .3f}, Recall:{val_recall_E: .3f}, Val F1: {val_f1_E: .3f} || Best F1: {val_best_f1_E: .3f}, Iter: {iteration_num: .0f}')

    train_result = [train_best_loss_E, train_best_accuracy_E, train_best_precision_E, train_best_recall_E, train_best_f1_E,
                    train_best_loss_S, train_best_accuracy_S, train_best_precision_S, train_best_recall_S, train_best_f1_S,
                    train_best_loss_G, train_best_accuracy_G, train_best_precision_G, train_best_recall_G, train_best_f1_G]
    val_result = [val_best_loss_E, val_best_accuracy_E, val_best_precision_E, val_best_recall_E, val_best_f1_E,
                  val_best_loss_S, val_best_accuracy_S, val_best_precision_S, val_best_recall_S, val_best_f1_S,
                  val_best_loss_G, val_best_accuracy_G, val_best_precision_G, val_best_recall_G, val_best_f1_G]

    return model, train_result, val_result, best_pred_list

In [ ]:
from google.colab import drive
drive.mount('/content/drive')